Cell codes copy-pasted from main.ipynb

In [50]:
import pandas as pd
import numpy as np

In [51]:
test = pd.read_csv('data/testset.csv', index_col=0, parse_dates=True)
test.head()

,bid_open,bid_high,bid_low,bid_close,ask_open,ask_high,ask_low,ask_close,bid_avg,ask_avg,spread_avg,tick_count,mid_avg,log_mid_returns
timestamp,,,,,,,,,,,,,,
2025-09-22 00:00:00,4.26516,4.26579,4.26515,4.26534,4.26769,4.2687,4.26725,4.2686,4.265448,4.267819,0.002371,111,4.266634,-0.000006
2025-09-22 00:01:00,4.26573,4.26573,4.26534,4.26534,4.26725,4.2686,4.26715,4.2680,4.265488,4.267666,0.002177,66,4.266577,-0.000013
2025-09-22 00:02:00,4.26556,4.26556,4.26534,4.26556,4.26720,4.2680,4.26720,4.2672,4.265453,4.267497,0.002044,33,4.266475,-0.000024
2025-09-22 00:03:00,4.26534,4.26611,4.26534,4.26570,4.26770,4.2684,4.26720,4.2684,4.265753,4.267780,0.002027,62,4.266767,0.000068
2025-09-22 00:04:00,4.26606,4.26606,4.26560,4.26560,4.26792,4.2684,4.26784,4.2684,4.265877,4.268109,0.002232,33,4.266993,0.000053


In [52]:
test['is_friday'] = (test.index.dayofweek == 4).astype(int)
test['is_sunday'] = (test.index.dayofweek == 6).astype(int)

In [53]:
# Wazne, aby najpierw przetransformowac raw prices features na log returns
prices_to_log = ['bid_open', 'bid_high', 'bid_low', 'bid_close', 'ask_open', 'ask_high',
       'ask_low', 'ask_close', 'bid_avg', 'ask_avg']
for feature in prices_to_log:
    test[f'log{feature}_returns'] = np.log(test[feature] / test[feature].shift(1))

# i pozbyc sie raw prices
test = test.drop(columns=['bid_open', 'bid_high', 'bid_low', 'bid_close', 'ask_open', 'ask_high',
       'ask_low', 'ask_close', 'bid_avg', 'ask_avg'])

test.columns

Index(['spread_avg', 'tick_count', 'mid_avg', 'log_mid_returns', 'is_friday',
       'is_sunday', 'logbid_open_returns', 'logbid_high_returns',
       'logbid_low_returns', 'logbid_close_returns', 'logask_open_returns',
       'logask_high_returns', 'logask_low_returns', 'logask_close_returns',
       'logbid_avg_returns', 'logask_avg_returns'],
      dtype='object')

In [54]:
# we already have log_mid_returns, so let's drop mid_avg
test = test.drop(columns=['mid_avg'])
test.columns

Index(['spread_avg', 'tick_count', 'log_mid_returns', 'is_friday', 'is_sunday',
       'logbid_open_returns', 'logbid_high_returns', 'logbid_low_returns',
       'logbid_close_returns', 'logask_open_returns', 'logask_high_returns',
       'logask_low_returns', 'logask_close_returns', 'logbid_avg_returns',
       'logask_avg_returns'],
      dtype='object')

In [55]:
# W sumie lagujemy wszustkie features poza dniem - intersuje nas tylko czy dzis jest 
# piatek badz niedziela, a nie co bylo wczoraj badz przedwczoraj
features_to_lag = ['spread_avg', 'tick_count', 'log_mid_returns',
       'logbid_open_returns', 'logbid_high_returns', 'logbid_low_returns',
       'logbid_close_returns', 'logask_open_returns', 'logask_high_returns',
       'logask_low_returns', 'logask_close_returns', 'logbid_avg_returns',
       'logask_avg_returns']

# How many minutes back?
lags = [1, 2, 3, 4,5,6,7,8,9,10]

# Create lagged features - po shiftnieciu calego datasetu bedzie to dylemat ktory mam w zeszycie
for feat in features_to_lag:
    for k in lags:
        test[f'{feat}_lag_{k}'] = test[feat].shift(k)

C:\Users\grzes\AppData\Local\Temp\ipykernel_27388\2619169018.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test[f'{feat}_lag_{k}'] = test[feat].shift(k)
C:\Users\grzes\AppData\Local\Temp\ipykernel_27388\2619169018.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test[f'{feat}_lag_{k}'] = test[feat].shift(k)
C:\Users\grzes\AppData\Local\Temp\ipykernel_27388\2619169018.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance. 

In [56]:
# To zapewne stworzylo kilka NA
# owszem. Usunmy je
test = test.dropna()

In [57]:
# Let's load the threshold. It was important to use the median computed from train subset, to prevent data leakage in the test set (median contains some sort of infomration about future minutes)
import json
with open("threshold.json", "r") as f:
    thr = json.load(f)["thr"]

In [59]:
# 1. Column with the log returns 3 minutes later
#    So the value for the first minute, will be the log return for the 4th minute and so on...
#    ATTENTION! Pracujemy na mid prices, a przeciez kupujemy za ASK, sprzedajemy po BID. Jest to slaba strona mojego projektu...
test['log_mid_return_3+'] = test['log_mid_returns'].shift(-3)

# 2. Define the threshold. let's take the absolute values of our log returns,
#    and compute the median. The predicted log return 3 periods later being higher than half
#    of the median will mean that we shouldnt stay flat
thr = test['log_mid_returns'].abs().median() * 0.5


# 3. Compute classification target
# - jesli predicted log return is bigger than threshold (+ values), lets go long
# - jesli predicted log return (bedacy zapwne minusowy) jest mniejszy niz liczba przeciwna 
#   do threshold, we go short
# - Otherwise, stay flat
# REMARK! I decided to penalize more the losses - that's why you can see -0.8 * thr
y = np.where(test['log_mid_return_3+'] > thr, 1,
    np.where(test['log_mid_return_3+'] < -0.8 * thr, -1, 0))

y = pd.Series(y, index=test.index, name='target')


# 4. - Prawie to mamy. Warto jeszcze jednak zauwazyc, ze ostatnie 5 minut przed zamknieciem gieldy (16:55-17:00) odpuscimy sobie tradowanie.
# - Warto tez zauwazyc, ze pierwsze 10 min w niedziele nie beda posiadac lagow. Ale mozemy
# to w sumie usunac prostym .droopna(), takze narazie sie tym nie przejmujemy
from datetime import time
idx = test.index

friday_last5 = (
    (idx.weekday == 4) &
    (idx.time >= time(16, 55)) &
    (idx.time <  time(17, 0))
)
y.loc[friday_last5] = 0

print(y)

timestamp
2025-09-22 00:11:00    0
2025-09-22 00:12:00    0
2025-09-22 00:13:00    0
2025-09-22 00:14:00   -1
2025-09-22 00:15:00   -1
                      ..
2025-10-05 23:55:00    0
2025-10-05 23:56:00    0
2025-10-05 23:57:00    0
2025-10-05 23:58:00    0
2025-10-05 23:59:00    0
Name: target, Length: 13513, dtype: int64


In [60]:
test['target'] = y

C:\Users\grzes\AppData\Local\Temp\ipykernel_27388\1695130833.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test['target'] = y


In [61]:
test[test.isna().any(axis=1)]

,spread_avg,tick_count,log_mid_returns,is_friday,is_sunday,logbid_open_returns,logbid_high_returns,logbid_low_returns,logbid_close_returns,logask_open_returns,...,logask_avg_returns_lag_3,logask_avg_returns_lag_4,logask_avg_returns_lag_5,logask_avg_returns_lag_6,logask_avg_returns_lag_7,logask_avg_returns_lag_8,logask_avg_returns_lag_9,logask_avg_returns_lag_10,log_mid_return_3+,target
timestamp,,,,,,,,,,,,,,,,,,,,,
2025-10-05 23:57:00,0.002510,14,2.220446e-16,0,1,0.000000,0.0,0.0,0.000000,0.000000,...,-0.000002,0.000002,0.000013,0.000027,-0.000028,-0.000018,0.000011,0.000004,NaN,0
2025-10-05 23:58:00,0.002502,25,-4.231253e-07,0,1,0.000000,0.0,0.0,0.000026,0.000000,...,0.000005,-0.000002,0.000002,0.000013,0.000027,-0.000028,-0.000018,0.000011,NaN,0
2025-10-05 23:59:00,0.002510,22,4.231253e-07,0,1,-0.000026,0.0,0.0,0.000000,0.000068,...,-0.000001,0.000005,-0.000002,0.000002,0.000013,0.000027,-0.000028,-0.000018,NaN,0


In [62]:
test = test.dropna()

In [63]:
test.isna().any().any()

np.False_

In [64]:
test = test.drop(columns= ['log_mid_return_3+'])

In [65]:
test.to_csv("algo_test_processed.csv")